In [123]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from pathlib import Path
import random

sys.path.append(os.path.abspath(os.path.join('../src')))
from varying_drift_diffusion import *
from motion_simulation import *
from accumulators import *

# bayesflow
sys.path.append(os.path.abspath(os.path.join('../../BayesFlow')))
from bayesflow.networks import InvariantNetwork, InvertibleNetwork
from bayesflow.amortizers import SingleModelAmortizer
from bayesflow.trainers import ParameterEstimationTrainer
from bayesflow.diagnostics import *

from tensorflow.keras.layers import Dense, GRU, LSTM, Conv1D, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential

In [124]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Constants

In [125]:
# simulation 
N_SIM = 500
N_OBS = 100

# bayesflow
PARAM_NAMES = ["a", "ndt", "bias", "kappa"]
N_PARAMS = len(PARAM_NAMES)
N_EPOCHS = 30
ITER_PER_EPOCH = 1000
BATCH_SIZE = 32
N_SAMPLES = 200

## Simulator Test

In [126]:
n_obs = 100
a     = 3.0
ndt   = 0.2
bias  = 0.5
kappa = 5
theta = np.array([a, ndt, bias, kappa])

unique_motions = np.array([-0.725, -0.675, -0.625, -0.575, -0.525, 0.525,  0.575,  0.625,  0.675,  0.725], dtype=np.float32)
amplitude = np.repeat(unique_motions, 10)
motion_set, condition = motion_experiment_manual(1, amplitude, 1)

rt, resp = var_dm_simulator(theta, 1, motion_set)


-0.0
-0.00010375285774677124
-0.0004149950472081053
-0.0009336774196342526
-0.0016597180690658506
-0.002593002345267822
-0.003733382871833983
-0.0050806795694594755
-0.006634679684377386
-0.008395137821955032
-0.010361775985444605
-0.012534283619882097
-0.01491231766112754
-0.017495502590038754
-0.020283430491770237
-0.02327566112018764
-0.026471721967387687
-0.029871108338312673
-0.033473283430447726
-0.03727767841858802
-0.04128369254466303
-0.04549069321260302
-0.04989801608823342
-0.0545049652041809
-0.059310813069774586
-0.06431480078592532
-0.06951613816496469
-0.0749140038554247
-0.0805075454717388
-0.0862958797288434
-0.09227809258165875
-0.09845323936942743
-0.1048203449648871
-0.11137840392825435
-0.11812638066599544
-0.1250632095943585
-0.13218779530764138
-0.13949901275116913
-0.1469957073989533
-0.15467669543600432
-0.1625407639452705
-0.1705866710991708
-0.1788131463556948
-0.18721889065903496
-0.19580257664472328
-0.20456284884923767
-0.2134983239240444
-0.22260759085404

In [113]:
p_, x_ = var_dm_batch_simulator(32, 100)
x_.shape

ValueError: could not broadcast input array from shape (10000,) into shape (1000,)

## Bayes Flow

In [ ]:
sum_meta = {
    'n_dense_s1': 2,
    'n_dense_s2': 3,
    'n_dense_s3': 2,
    'n_equiv':    2,
    'dense_s1_args': {'activation': 'relu', 'units': 32},
    'dense_s2_args': {'activation': 'relu', 'units': 64},
    'dense_s3_args': {'activation': 'relu', 'units': 32},
}

summary_net = InvariantNetwork()

In [ ]:
# invertable inference network
meta_dict={
    'n_coupling_layers': 5,
    's_args': {
        'units': [128, 128],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    't_args': {
        'units': [128, 128],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    'alpha': 1.9,
    'permute': True,
    'use_act_norm': True,
    'n_params': N_PARAMS
}

inference_net = InvertibleNetwork(meta_dict)

# Connect summary and inference network
amortizer = SingleModelAmortizer(inference_net, summary_net)

In [ ]:
# Learning-rate decay
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0005, 1000, 0.99, staircase=True
)

trainer = ParameterEstimationTrainer(
    network=amortizer, 
    generative_model=var_dm_batch_simulator,
    learning_rate=learning_rate,
    checkpoint_path='../src/selected_checkpoints/time_var_dm',
    clip_value=3,
    max_to_keep=5
)

In [ ]:
%%time
# online training
losses = trainer.train_online(N_EPOCHS, ITER_PER_EPOCH, BATCH_SIZE, n_obs=N_OBS)

## Parameter Recovery

In [ ]:
# Simulate and amortized inference
p_, x_ = var_dm_batch_simulator(n_sim=N_SIM,n_obs=N_OBS)
samples = amortizer.sample(x_, n_samples=N_SAMPLES)
param_means = samples.mean(axis=1)

In [ ]:
# Recovery plot
true_vs_estimated(theta_true=p_, theta_est=param_means,
                  param_names=PARAM_NAMES, dpi=300, figsize=(20,6),font_size=16)

In [ ]:
sns.pairplot(pd.DataFrame(samples[0], columns=PARAM_NAMES))

## Simulation Based Calibration

In [ ]:
# Simulate
n_sbc = 5000
n_post_samples_sbc = 250
params, sim_data = var_dm_batch_simulator(n_sbc, N_OBS)

In [ ]:
# Amortized inference
param_samples = np.concatenate([amortizer.sample(x, n_post_samples_sbc)
                                for x in tf.split(sim_data, 10, axis=0)], axis=0)

In [ ]:
# Rank-plot
f = plot_sbc(param_samples, params, param_names=PARAM_NAMES, figsize=(24, 8))

## Bayesian Eye Chart

In [ ]:
# Simulation
true_params, sim_data = var_dm_batch_simulator(N_SIM, N_OBS)

# Amortized inference
param_samples = np.concatenate([amortizer.sample(x, N_SAMPLES)
                                for x in tf.split(sim_data, 10, axis=0)], axis=0)

In [ ]:
### Posterior z-score
# Compute posterior means and stds
post_means = param_samples.mean(1)
post_stds = param_samples.std(1)
post_vars = param_samples.var(1)

# Compute posterior z score
post_z_score = (post_means - true_params) / post_stds

### Posterior contraction, i.e., 1 - post_var / prior_var
prior_a = (0.5, 0.1, 0.0) # lower bound of uniform prior
prior_b = (3.0, 0.5, 3.0) # upper bound of uniform prior

# Compute prior vars analytically
prior_vars = np.array([(b-a)**2/12 for a,b in zip(prior_a, prior_b)])
prior_vars = np.concatenate((prior_vars[0:2], np.array([0.0025]), prior_vars[2:]))
post_cont = 1 - post_vars / prior_vars

# Plotting time
f, axarr = plt.subplots(1, 4, figsize=(16, 4))
for i, (p, ax) in enumerate(zip(PARAM_NAMES, axarr.flat)):


    ax.scatter(post_cont[:, i], post_z_score[:, i], color='#8f2727', alpha=0.7)
    ax.set_title(p, fontsize=20)
    sns.despine(ax=ax)
    ax.set_xlim([-0.1, 1.05])
    ax.set_ylim([-3.5, 3.5])
    ax.grid(color='black', alpha=0.1)
    ax.set_xlabel('Posterior contraction', fontsize=14)
    if i == 0 or i == 3:
        ax.set_ylabel('Posterior z-score', fontsize=14)
f.tight_layout()

In [ ]:
# empirical data preparation
directory = str(Path().absolute())
path = str(Path(directory).parents[1]) + '/evidence-accumulators/data/single_sub_data.csv'
# data = np.loadtxt(open(path, 'rb'), delimiter=",", skiprows=1)
data = pd.read_csv(path)

In [ ]:
idx = np.where((data["condition"] == 1) & (data["instruction"] == 1))
data = data.loc[idx]
data.head()

In [ ]:
amplitude = data["motion"]
frequency = data["motion_duration"][0]
motion_dur = data["motion_duration"][0]

motion_set, condition = motion_experiment_manual(motion_dur, amplitude, frequency)


In [ ]:
directory = str(Path().absolute())
path = str(Path(directory).parents[1]) + '/evidence-accumulators/data/single_sub_data.csv'
# data = np.loadtxt(open(path, 'rb'), delimiter=",", skiprows=1)
data = pd.read_csv(path)
idx = np.where((data["condition"] == 1) & (data["instruction"] == 1))
data = data.loc[idx]
amplitude = data["motion"]
frequency = data["motion_duration"][0]
motion_dur = data["motion_duration"][0]

motion_set, condition = motion_experiment_manual(motion_dur, amplitude, frequency)

In [ ]:
unique_motions = np.array([-0.725, -0.675, -0.625, -0.575, -0.525,
                           0.525,  0.575,  0.625,  0.675,  0.725], dtype=np.float32)

In [ ]:
 x_ = np.repeat(unique_motions, 10)

In [ ]:
motion_set, condition = motion_experiment_manual(motion_dur, x_, frequency)

In [ ]:
motion_set

In [ ]:
motion_set[np.random.choice(motion_set.shape[0], motion_set.shape[0], replace=False)]

In [ ]:
# get exact motion profile used in the data
directory = str(Path().absolute())
path = str(Path(directory).parents[1]) + '/evidence-accumulators/data/single_sub_data.csv'
# data = np.loadtxt(open(path, 'rb'), delimiter=",", skiprows=1)
data = pd.read_csv(path)
idx = np.where((data["condition"] == 1) & (data["instruction"] == 1))
data = data.loc[idx]
amplitude = data["motion"]
frequency = data["motion_duration"][0]
motion_dur = data["motion_duration"][0]

motion_set, condition = motion_experiment_manual(motion_dur, amplitude, frequency)

In [ ]:
unique_motions = np.array([-0.725, -0.675, -0.625, -0.575, -0.525,
                           0.525,  0.575,  0.625,  0.675,  0.725], dtype=np.float32)
amplitude = np.repeat(unique_motions, 10)
motion_set, condition = motion_experiment_manual(1, amplitude, 1)

In [ ]:
rng = np.random.default_rng(2021)
rng.uniform(0.5, 5, size=5)
rng.uniform(0.5, 5, size=5)

In [ ]:
unique_motions = np.array([-0.725, -0.675, -0.625, -0.575, -0.525,
                           0.525,  0.575,  0.625,  0.675,  0.725], dtype=np.float32)
amplitude = np.repeat(unique_motions, 10)
motion_set, condition = motion_experiment_manual(1, amplitude, 1)

In [ ]:
shuffler = np.random.permutation(len(motion_set))

In [ ]:
motion_set[shuffler, :]
condition[shuffler, :]

In [ ]:
n_unique_motions = 5
motion_dur = 1
motion_set, condition = motion_experiment(n_obs, n_unique_motions, motion_dur)
motion_set.shape

In [ ]:
unique_motions = np.array([-0.725, -0.675, -0.625, -0.575, -0.525,
                       0.525,  0.575,  0.625,  0.675,  0.725], dtype=np.float32)
amplitude = np.repeat(unique_motions, 10)
motion_set, condition = motion_experiment_manual(1, amplitude, 1)
motion_set.shape

In [ ]:
n_unique_motions = 5
motion_dur = 1
motion_set, condition = motion_experiment(n_obs, n_unique_motions, motion_dur)
motion_set.shape
condition

In [ ]:
unique_motions = np.array([-0.725, -0.675, -0.625, -0.575, -0.525,
                       0.525,  0.575,  0.625,  0.675,  0.725], dtype=np.float32)
amplitude = np.repeat(unique_motions, 10)
motion_set, condition = motion_experiment_manual(1, amplitude, 1)
motion_set.shape
condition

In [ ]:
shuffler = np.random.permutation(len(motion_set))
motion_set = motion_set[shuffler, :]
condition = condition[shuffler, :]

In [ ]:
condition

In [ ]:
unique_motions = np.array([-0.725, -0.675, -0.625, -0.575, -0.525, 0.525,  0.575,  0.625,  0.675,  0.725], dtype=np.float32)
amplitude = np.repeat(unique_motions, 10)
motion_set, condition = motion_experiment_manual(1, amplitude, 1)

In [ ]:
condition[0:100, :].shape